In [1]:
import pickle
import numpy as np 
import pandas as pd 
import multiprocessing 
from pathlib import Path 
from scipy.stats import norm
from numpy.linalg import inv
from settings import settings 
from itertools import product
from joblib import Parallel, delayed

## Define Parameter

In [2]:
data_path = Path("result")

In [3]:
cluster_labels = pd.read_parquet(data_path / "cluster_labels.parquet")

In [4]:
with open(data_path / "eb_est.pkl", "rb") as f:
    eb_est = pickle.load(f)

## Simulations EB vs BY

In [5]:
pairwise_cor = eb_est['us']['input']['long'].pivot_table(index='eom', columns='characteristic', values='ret_neu').corr(method='pearson')

In [6]:
pairwise_cor = pairwise_cor.stack().to_frame("cor")
pairwise_cor.index.names = ["level_0", "level_1"]
cor_value = pairwise_cor.reset_index().copy()

In [7]:
# 각각 factor별 cluster 이름 merge
cor_value = cor_value.merge(cluster_labels[['characteristic', 'hcl_label']].rename(columns={'hcl_label': 'hcl1'}), left_on='level_0', right_on='characteristic', how='left')
cor_value = cor_value.merge(cluster_labels[['characteristic', 'hcl_label']].rename(columns={'hcl_label': 'hcl2'}), left_on='level_1', right_on='characteristic', how='left')

In [8]:
cor_value['same_cluster'] = (cor_value['hcl1'] == cor_value['hcl2'])

In [9]:
cor_value_avg = cor_value.groupby('same_cluster')['cor'].mean().reset_index(name='avg_cor')

In [10]:
# 같은 cluster와 다른 cluster간에 상관관계 평균
cor_value_avg = cor_value.groupby('same_cluster')['cor'].mean().reset_index(name='avg_cor')

### Time Periods

In [11]:
med_months = eb_est['us']['input']['long'].groupby('characteristic')['eom'].count().median()

In [12]:
data_sim = {
    'yrs': round(med_months / 12),
    'cor_within': round(cor_value_avg[cor_value_avg['same_cluster'] == True]['avg_cor'].values[0], 2),
    'cor_across': round(cor_value_avg[cor_value_avg['same_cluster'] == False]['avg_cor'].values[0], 2)
}

In [13]:
# Set seed
np.random.seed(settings['seed'])

In [14]:
tau_c_list = [0.01] + list(np.arange(0.05, 0.55, 0.05))
tau_w_list =  [0.01, 0.2]

sim = {
    # alpha는 0으로 가정
    "alpha_0": 0,
    "t": 12 * 70,
    "clusters": 13,
    # cluster 안에 요인 개수 
    "fct_pr_cl": 10,
    "corr_within": 0.58,
    "corr_across": 0.02,
    "n_sims": 10000,
}
sim['se'] = (10 / np.sqrt(12)) / np.sqrt(sim['t'])
sim['n'] = sim['clusters'] * sim['fct_pr_cl']

In [15]:
iter_list = list(product(tau_c_list, tau_w_list))

### Simulation Multiple Testing Control

In [16]:
def sim_mt_control(tau_c, tau_w, sim_settings):
    # Cluster membership matrix
    m = np.zeros((sim_settings['n'], sim_settings['clusters']))
    j = 0

    # 130개 요인 X 13개 Cluster
    for i in range(sim_settings['clusters']):
        m[j:j + sim_settings['fct_pr_cl'], i] = 1
        j += sim_settings['fct_pr_cl']

    
    
    # 같은 클러스터에 속하는 요인들의 matrix를 임시로 correlation matrix로 사용
    corr_mat = m @ m.T
    # 다른 Factor 클러스터와 상관관계는 corr_across로 대체하고 
    corr_mat[corr_mat == 0] = sim_settings['corr_across']
    # 같은 Factor 클러스터와 상관관계는 corr_within 대체함
    corr_mat[corr_mat == 1] = sim_settings['corr_within']
    np.fill_diagonal(corr_mat, 1)
        
    # Sigma matrix
    sigma = (sim_settings['se'] ** 2) * corr_mat
    
    # Predefine variables
    # 요인 개수만큼 생성 
    alpha_0_vec = np.full(sim_settings['n'], sim_settings['alpha_0'])

    i_n = np.eye(sim_settings['n'])
    
    results = []
    
        
    # Preallocate alpha noise
    alpha_noise = np.random.multivariate_normal(mean=np.zeros(sim_settings['n']), cov=sigma, size=sim_settings['n_sims'])
    
    sim_results = []
    
    for s in range(sim_settings['n_sims']):
        # Omega matrix
        omega = np.dot(m, m.T) * (tau_c ** 2) + i_n * (tau_w ** 2)
        
        # True alphas
        alpha_c = np.random.normal(0, tau_c, sim_settings['clusters'])
        alpha_w = np.random.normal(0, tau_w, sim_settings['n'])
        alpha_true = alpha_0_vec + np.dot(m, alpha_c) + alpha_w
        alpha_hat = alpha_true + alpha_noise[s]
        
        # Posterior variance and alpha
        post_var = inv(inv(omega) + inv(sigma))
        post_alpha = np.dot(post_var, np.dot(inv(omega), alpha_0_vec) + np.dot(inv(sigma), alpha_hat))
        
        # Empirical Bayes results
        post_alpha_z = post_alpha / np.sqrt(np.diag(post_var))
        post_alpha_p = 2 * norm.sf(np.abs(post_alpha_z))
        eb = pd.DataFrame({
            'type': 'eb',
            'true_alpha': alpha_true,
            'z': post_alpha_z,
            'p': post_alpha_p
        })
        
        # OLS results
        ols_z = alpha_hat / np.sqrt(np.diag(sigma))
        ols_p = 2 * norm.sf(np.abs(ols_z))
        ols = pd.DataFrame({
            'type': 'ols',
            'true_alpha': alpha_true,
            'z': ols_z,
            'p': ols_p
        })
        
        # Benjamini-Yekutieli correction
        by = pd.DataFrame({
            'type': 'by',
            'true_alpha': alpha_true,
            'z': ols_z,
            'p': pd.Series(ols_p).transform(lambda x: pd.Series(x).rank(ascending=False) / len(x))  # BY p-value adjustment placeholder
        })
        
        # Combine results
        all_results = pd.concat([eb, ols, by], ignore_index=True)
        all_results['sig'] = (all_results['z'] > 0) & (all_results['p'] < 0.025)
        
        summary = all_results.groupby('type').apply(lambda df: pd.Series({
            'sim': s,
            'n_disc': df['sig'].sum(),
            'true_disc': (np.sign(df['true_alpha'][df['sig']]) == np.sign(df['z'][df['sig']])).sum(),
            'false_disc': df['sig'].sum() - (np.sign(df['true_alpha'][df['sig']]) == np.sign(df['z'][df['sig']])).sum()
        }), include_groups=False)
        sim_results.append(summary)
    
    sim_results_df = pd.concat(sim_results)
    summary_results = sim_results_df.groupby('type').apply(lambda df: pd.Series({
        'fdr': np.mean(df['false_disc'] / df['n_disc']),
        'n_disc': np.mean(df['n_disc']),
        'false_disc': np.mean(df['false_disc']),
        'true_disc': np.mean(df['true_disc']),
        'tau_c': tau_c,
        'tau_w': tau_w,
        'n': len(df)
    }))
    summary_results['true_disc_rate'] = summary_results['true_disc'] / (sim_settings['n'] / 2)
    results.append(summary_results)
    
    return pd.concat(results)

### Simulate Parallel

In [17]:
n_cpus = int( multiprocessing.cpu_count() * 2 / 3)
parallel = Parallel(n_jobs=n_cpus, verbose=True)

In [18]:
sim_result_list = parallel(delayed(sim_mt_control)(tau_c=i[0], tau_w=i[1], sim_settings=sim) for i in iter_list)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


[Parallel(n_jobs=10)]: Done  22 out of  22 | elapsed:  2.6min finished


In [20]:
simulation = pd.concat(sim_result_list)

In [21]:
simulation.to_csv(data_path / "fdr_sim.csv")